# Fine-Tuning of CNN models

In order to transfer learning from a pre-trained models available in [Keras](https://keras.io/api/applications/) to solve specific problems, this project shows how to apply a famous machine learning technique called **fine-tuning**.

**Problem: [Domestic Garbage Detection](https://www.kaggle.com/datasets/farzadnekouei/trash-type-image-dataset/)**

## Steps
* Import the dataset;
* Exploration and preprocessing
* Compare 3 (or more) pre-trained models
* Evaluating the best model

## Importig libraries

In [ ]:
import os
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras_tuner as kt

from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
from datetime import datetime

## Defining general constants

In [ ]:
BATCH_SIZE=32
IMAGE_SIZE=(384, 512)
EPOCHS=1
SEED=10 

CURRENT_PATH = os.getcwd()
DATASET_PATH = os.path.join(CURRENT_PATH, 'datasets', 'trash_type_dataset')
TRAIN_PATH = os.path.join(DATASET_PATH, 'train')
VALIDATION_PATH = os.path.join(DATASET_PATH, 'validation')
TEST_PATH = os.path.join(DATASET_PATH, 'test')

## Importing dataset

In [ ]:
def import_dataset(dataset_path: str, batch_size: int = None):
    dataset = tf.keras.utils.image_dataset_from_directory(dataset_path,
                                                        labels='inferred',
                                                        label_mode='int',
                                                        batch_size=batch_size,
                                                        image_size=IMAGE_SIZE)
    return dataset

def import_datasets():
    print(f"[INFO] Importing training dataset from {TRAIN_PATH}")
    train_dataset = import_dataset(TRAIN_PATH, BATCH_SIZE)

    print(f"[INFO] Importing validation dataset from {VALIDATION_PATH}")
    validation_dataset = import_dataset(VALIDATION_PATH, BATCH_SIZE)

    print(f"[INFO] Importing test dataset from {TEST_PATH}")
    test_dataset = import_dataset(TEST_PATH, batch_size=None)

    return train_dataset, validation_dataset, test_dataset

In [ ]:
train_dataset, validation_dataset, test_dataset = import_datasets()

## Exploring the dataset

What are the classes?

In [ ]:
classes = train_dataset.class_names
print(f"The classes are {classes}")

Preview of the samples

In [ ]:
def get_sample_of_dataset(dataset):
    image, label = next(iter(dataset))
    return image[0].numpy().astype(np.uint8), classes[label[0].numpy()]

def plot_n_samples(dataset, n: int, ncols = 4):
    samples = []
    nrows = (n // ncols) + 1
    figure, axis = plt.subplots(ncols=ncols, nrows=nrows, figsize=(12,8))

    figure.suptitle("Samples - Garbage Image Classification")

    for _ in range(len(samples)):
        samples.append(get_sample_of_dataset(dataset))

    for ax in axis.reshape(-1):
        image, label = get_sample_of_dataset(dataset)
        ax.set_title(f"[class={label}]")
        ax.imshow(image)
        ax.axis("off")

    plt.subplots_adjust(wspace=0)

In [ ]:
plot_n_samples(train_dataset, 10)

TODO: create a new notebook just for exploration step

## Preparing experiments
For this project, some [available models](https://keras.io/api/applications/#available-models) were chosen to be compared using [Keras](https://keras.io/) framework. Below, it is described each used model:

|Model|Size (MB)|Top-1 Accuracy (ImageNet)|
|---|---|---|
|EfficientNetB7|256|84.3%|
|EfficientNetV2S|88|83.9%|
|ConvNeXtXLarge|1310|86.7%|

Before start the experiments, let's define some configurations and functions in order to re-use funcionalities.

In [ ]:
class FineTuningModel:
    def __init__(self, 
                 title: str, 
                 base_model, 
                 input_shape = IMAGE_SIZE, 
                 batch_size = BATCH_SIZE):
        self.title = title
        self.base_model = base_model
        self.input_shape = input_shape
        self.batch_size = batch_size
        self.best_model = None
        self._freeze_base_model()

    def _freeze_base_model(self):
        self.base_model.trainable = False

    def _build(self, hp: kt.HyperParameters):
        dense_units = hp.Choice('dense_units', values=[256, 128, 64])
        
        # Conv Layers using a pre-trained model
        inputs = tf.keras.Input(shape=(*self.input_shape, 3))
        x = self.base_model(inputs, training=False)
        
        # Dense layers
        x = tf.keras.layers.GlobalAveragePooling2D()(x)
        x = tf.keras.layers.Dropout(0.2)(x)
        x = tf.keras.layers.Dense(dense_units, activation='relu')(x)
        outputs = tf.keras.layers.Dense(len(classes), activation='softmax')(x)
        
        # Compiling the model
        model = tf.keras.Model(inputs, outputs)
        model = self._compile(model, hp)
        return model

    def _compile(self, model: tf.keras.Model, hp: kt.HyperParameters):
        learning_rate = hp.Choice('learning_rate', values=[0.01, 0.001])
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                      loss='sparse_categorical_crossentropy',
                      metrics=[tf.keras.metrics.CategoricalAccuracy(name='accuracy')])
        return model

    def _get_best_model(self, train, validation, best_hps: list):
        best_hp = best_hps[0]
        model = self._build(best_hp)
        model.fit(train, validation_data=validation, epoch=EPOCHS)
        return model

    def train(self, train: tf.data.Dataset, validation: tf.data.Dataset):
        tuner = kt.Hyperband(hypermodel=self._build,
                            objective=kt.Objective('val_accuracy', direction='max'),
                            overwrite=True,
                            directory='checkpoints',
                            project_name=self.title,
                            max_epochs=EPOCHS)
        
        tuner.search(train, 
                     batch_size=BATCH_SIZE, 
                     validation_data=validation)
        self.best_model = self._get_best_model(train,
                                                validation,
                                                tuner.get_best_hyperparameters(5))

Defining the experiments

In [ ]:
experiments = [
    FineTuningModel('EfficientNetB7', 
                    tf.keras.applications.EfficientNetB7(include_top=False)),
    FineTuningModel('EfficientNetV2S', 
                    tf.keras.applications.EfficientNetV2S(include_top=False, input_shape=(*IMAGE_SIZE, 3))),
    FineTuningModel('ConvNeXtXLarge', 
                    tf.keras.applications.ConvNeXtXLarge(include_top=False))
]

## Training

In [ ]:
for experiment in experiments:
    experiment.train(train=train_dataset, validation=validation_dataset)

## Evaluating the best experiment

## Testing

## Conclusion